In [1]:
from google.colab import files

uploaded = files.upload()


Saving DataEngineeringQ2.json to DataEngineeringQ2.json


In [2]:
import json

# Get the uploaded file name
file_name = list(uploaded.keys())[0]

# Read the JSON file
with open(file_name, 'r') as file:
    data = json.load(file)


In [3]:
# Function to calculate percentage of missing values
def calculate_missing_percentage(data, columns):
    total_records = len(data)
    missing_counts = {column: 0 for column in columns}

    for record in data:
        patient_details = record.get('patientDetails', {})
        for column in columns:
            if not patient_details.get(column) or patient_details[column] == "":
                missing_counts[column] += 1

    missing_percentages = {column: (count / total_records) * 100 for column, count in missing_counts.items()}
    return {column: round(percentage, 2) for column, percentage in missing_percentages.items()}

# Columns to check for missing values
columns_to_check = ["firstName", "lastName", "birthDate"]

# Calculate missing percentages
missing_percentages = calculate_missing_percentage(data, columns_to_check)

# Print the results
print(missing_percentages)


{'firstName': 0.0, 'lastName': 70.97, 'birthDate': 32.26}


In [4]:
from collections import Counter

def impute_gender_with_mode(data):
    genders = [record['patientDetails'].get('gender') for record in data if record['patientDetails'].get('gender')]
    mode_gender = Counter(genders).most_common(1)[0][0]

    for record in data:
        if not record['patientDetails'].get('gender'):
            record['patientDetails']['gender'] = mode_gender

    return data, mode_gender

# Impute missing gender values with the mode
data, mode_gender = impute_gender_with_mode(data)

print(f"Mode gender is: {mode_gender}")


Mode gender is: M


In [5]:
def calculate_female_percentage(data):
    total_records = len(data)
    female_count = sum(1 for record in data if record['patientDetails'].get('gender') == 'female')

    female_percentage = (female_count / total_records) * 100
    return round(female_percentage, 2)

# Calculate the percentage of female gender
female_percentage = calculate_female_percentage(data)

print(f"Percentage of female gender after imputation: {female_percentage}%")


Percentage of female gender after imputation: 0.0%


In [11]:
from datetime import datetime
def calculate_age(birthDate):
    if birthDate:
        # Handle the full datetime format
        birth_date = datetime.strptime(birthDate, "%Y-%m-%dT%H:%M:%S.%fZ")
        today = datetime.today()
        return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return None

def categorize_age_group(age):
    if age is not None:
        if age <= 12:
            return "Child"
        elif age <= 19:
            return "Teen"
        elif age <= 59:
            return "Adult"
        else:
            return "Senior"
    return "Unknown"

def add_age_group(data):
    for record in data:
        birthDate = record['patientDetails'].get('birthDate')
        age = calculate_age(birthDate)
        age_group = categorize_age_group(age)
        record['patientDetails']['ageGroup'] = age_group
    return data

# Add age group to each patient record
data = add_age_group(data)


In [12]:
def count_age_group(data, age_group):
    return sum(1 for record in data if record['patientDetails'].get('ageGroup') == age_group)

# Count the number of adults
adult_count = count_age_group(data, "Adult")

print(f"Count of Adult: {adult_count}")


Count of Adult: 21


In [13]:
def calculate_average_medicines(data):
    total_medicines = 0
    total_patients = len(data)

    for record in data:
        medicines = record['consultationData'].get('medicines', [])
        total_medicines += len(medicines)

    if total_patients == 0:
        return 0

    average_medicines = total_medicines / total_patients
    return round(average_medicines, 2)

# Calculate the average number of medicines prescribed
average_medicines = calculate_average_medicines(data)

print(f"Average number of medicines prescribed: {average_medicines}")


Average number of medicines prescribed: 2.13


In [14]:
from collections import Counter

def count_medicine_frequencies(data):
    medicine_counter = Counter()

    for record in data:
        medicines = record['consultationData'].get('medicines', [])
        for medicine in medicines:
            medicine_name = medicine.get('medicineName')
            if medicine_name:
                medicine_counter[medicine_name] += 1

    return medicine_counter

# Count the frequencies of each medicine name
medicine_counter = count_medicine_frequencies(data)


In [15]:
def get_nth_most_frequent_medicine(medicine_counter, n):
    most_common_medicines = medicine_counter.most_common()
    if len(most_common_medicines) >= n:
        return most_common_medicines[n-1][0]  # n-1 because list indices start at 0
    return None

# Get the 3rd most frequently prescribed medicine
third_most_frequent_medicine = get_nth_most_frequent_medicine(medicine_counter, 3)

print(f"The 3rd most frequently prescribed medicine is: {third_most_frequent_medicine}")


The 3rd most frequently prescribed medicine is: C


In [16]:
def count_medicine_status(data):
    active_count = 0
    inactive_count = 0

    for record in data:
        medicines = record['consultationData'].get('medicines', [])
        for medicine in medicines:
            if medicine.get('isActive'):
                active_count += 1
            else:
                inactive_count += 1

    return active_count, inactive_count

# Count the number of active and inactive medicines
active_count, inactive_count = count_medicine_status(data)


In [17]:
def calculate_percentage_distribution(active_count, inactive_count):
    total_count = active_count + inactive_count

    if total_count == 0:
        return 0, 0

    active_percentage = (active_count / total_count) * 100
    inactive_percentage = (inactive_count / total_count) * 100

    return round(active_percentage, 2), round(inactive_percentage, 2)

# Calculate the percentage distribution
active_percentage, inactive_percentage = calculate_percentage_distribution(active_count, inactive_count)

print(f"Percentage of active medicines: {active_percentage}%")
print(f"Percentage of inactive medicines: {inactive_percentage}%")


Percentage of active medicines: 69.7%
Percentage of inactive medicines: 30.3%


In [18]:
def is_valid_indian_phone_number(phone_number):
    # Remove spaces and any unwanted characters
    phone_number = phone_number.replace(' ', '').strip()

    # Check for valid prefixes
    if phone_number.startswith('+91'):
        phone_number = phone_number[3:]
    elif phone_number.startswith('91'):
        phone_number = phone_number[2:]

    # Check if the number is exactly 10 digits long and in the valid range
    if len(phone_number) == 10 and phone_number.isdigit():
        number = int(phone_number)
        return 6000000000 <= number <= 9999999999

    return False


In [19]:
def add_valid_mobile_column(data):
    for record in data:
        phone_number = record.get('phoneNumber', '')
        record['isValidMobile'] = is_valid_indian_phone_number(phone_number)
    return data

# Add the isValidMobile column to each record
data = add_valid_mobile_column(data)


In [20]:
def count_valid_phone_numbers(data):
    return sum(1 for record in data if record.get('isValidMobile', False))

# Count the number of valid phone numbers
valid_phone_count = count_valid_phone_numbers(data)

print(f"Number of valid phone numbers: {valid_phone_count}")


Number of valid phone numbers: 18


In [22]:
from datetime import datetime
def calculate_age(birth_date):
    # Remove time and timezone information from birth_date string
    if 'T' in birth_date:
        birth_date = birth_date.split('T')[0]

    # Parse the date
    birth_date = datetime.strptime(birth_date, "%Y-%m-%d")
    today = datetime.today()
    return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))

# Function to extract ages and number of prescribed medicines
def extract_data(data):
    ages = []
    num_medicines = []

    for record in data:
        # Get age
        birth_date = record.get('patientDetails', {}).get('birthDate', None)
        if birth_date:
            age = calculate_age(birth_date)
            ages.append(age)
        else:
            ages.append(np.nan)  # Handle missing age

        # Get number of prescribed medicines
        medicines = record['consultationData'].get('medicines', [])
        num_medicines.append(len(medicines))

    return np.array(ages), np.array(num_medicines)

# Function to calculate Pearson correlation coefficient
def calculate_pearson_correlation(ages, num_medicines):
    # Remove missing values
    valid_indices = ~np.isnan(ages)
    ages = ages[valid_indices]
    num_medicines = num_medicines[valid_indices]

    if len(ages) == 0:
        return np.nan

    correlation = np.corrcoef(ages, num_medicines)[0, 1]
    return round(correlation, 2)

# Extract data
ages, num_medicines = extract_data(data)

# Calculate Pearson correlation
pearson_correlation = calculate_pearson_correlation(ages, num_medicines)

print(pearson_correlation)

-0.22
